---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [107]:
House = pd.read_csv('City_Zhvi_AllHomes.csv')
House.replace(states, inplace=True)
House.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,New York,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,California,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,Illinois,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,Pennsylvania,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,Arizona,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [108]:
df = pd.read_fwf('university_towns.txt')
df.columns = ["State"]
States =[]
Regions=[]
new='Alabama'
for i in range(0, len(df)):
    if 'edit' in df.iloc[i]['State']:
        df.iloc[i]['State'] = df.iloc[i]['State'].split('[')[0]
        new = df.iloc[i]['State']
    elif 'edit' not in df.iloc[i]['State']:
        df.iloc[i]['State'] = df.iloc[i]['State'].split('(')[0]
        Regions.append(df.iloc[i]['State'])
        States.append(new)
for i in range(0, len(Regions)):
    if Regions[i][-1] == ' ':
        Regions[i] = Regions[i][0:-1]
        
        
len(States)

517

In [109]:
len(Regions)

517

In [110]:
len(df)

566

In [111]:
df = pd.DataFrame(columns=['State', 'RegionName'])
df['State']= States
df['RegionName'] = Regions
#df['State'].replace({'Alabama':'Al', axis=0, inplace=True)
df.shape

(517, 2)

In [112]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    return df
df

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [113]:
GDP = pd.read_excel('gdplev.xls')
GDP.columns = GDP.iloc[4]
GDP = GDP.iloc[7:len(GDP)]
GDP

4,nan,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,nan,nan,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,nan
7,1929,104.6,1056.6,NaN,1947q1,243.1,1934.5,NaN
8,1930,92.2,966.7,NaN,1947q2,246.3,1932.3,NaN
9,1931,77.4,904.8,NaN,1947q3,250.1,1930.3,NaN
10,1932,59.5,788.2,NaN,1947q4,260.3,1960.7,NaN
11,1933,57.2,778.3,NaN,1948q1,266.2,1989.5,NaN
12,1934,66.8,862.2,NaN,1948q2,272.9,2021.9,NaN
13,1935,74.3,939,NaN,1948q3,279.5,2033.2,NaN
14,1936,84.9,1060.5,NaN,1948q4,280.7,2035.3,NaN
15,1937,93,1114.6,NaN,1949q1,275.4,2007.5,NaN
16,1938,87.4,1077.7,NaN,1949q2,271.7,2000.8,NaN


In [114]:
GDP.columns = ['a', 'b', 'c', 'd', 'Q', 'gdp', 'f', 'g']

In [115]:
GDP['Q'].astype(str)
def spl(a):
    return int(a['Q'].split('q')[0])
    
GDP['Year'] = GDP.apply(spl, axis=1)

GDP= GDP[GDP['Year']>= 2000]
GDP.iloc[30:]

,a,b,c,d,Q,gdp,f,g,Year
249,NaN,NaN,NaN,NaN,2007q3,14569.7,14938.5,NaN,2007
250,NaN,NaN,NaN,NaN,2007q4,14685.3,14991.8,NaN,2007
251,NaN,NaN,NaN,NaN,2008q1,14668.4,14889.5,NaN,2008
252,NaN,NaN,NaN,NaN,2008q2,14813,14963.4,NaN,2008
253,NaN,NaN,NaN,NaN,2008q3,14843,14891.6,NaN,2008
254,NaN,NaN,NaN,NaN,2008q4,14549.9,14577,NaN,2008
255,NaN,NaN,NaN,NaN,2009q1,14383.9,14375,NaN,2009
256,NaN,NaN,NaN,NaN,2009q2,14340.4,14355.6,NaN,2009
257,NaN,NaN,NaN,NaN,2009q3,14384.1,14402.5,NaN,2009
258,NaN,NaN,NaN,NaN,2009q4,14566.5,14541.9,NaN,2009


In [116]:
Rec_start =''
Rec_end = ''
for i in range(0, len(GDP)-2):
    if GDP['f'].iloc[i] > GDP['f'].iloc[i+1]:
        if GDP['f'].iloc[i+1] > GDP['f'].iloc[i+2]:
            Rec_start = GDP['Q'].iloc[i+1]
            break

Rec_start
GDP2 = GDP[GDP['Year']>= 2008]
GDP2

for i in range(0, len(GDP2)-2):
    if GDP2['f'].iloc[i] < GDP2['f'].iloc[i+1]:
        if GDP2['f'].iloc[i+1] < GDP2['f'].iloc[i+2]:
            Rec_end = GDP2['Q'].iloc[i+2]
            break
Rec_end

'2009q4'

In [117]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return Rec_start
get_recession_start()

'2008q3'

In [118]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    return Rec_end

get_recession_end()

'2009q4'

In [119]:
def buttom(a):
    b = 14843
    c=''
    for i in range(2008, 2010):
        if i == 2008:
            for k in range(3,5):
                if a['Q'] == str(i) + 'q'+ str(k):
                    if a['f'] <= b:
                        b= a['f']
                        c= a['Q']
        elif i ==2009:
            for k in range (1,5):
                if a['Q'] == str(i) + 'q'+ str(k):
                    if a['f'] <= b:
                        b= a['f']
                        c= a['Q']
    return b,c

min(GDP.apply(buttom,axis=1))[1]

'2009q2'

In [120]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return min(GDP.apply(buttom,axis=1))[1]

get_recession_bottom()

'2009q2'

In [121]:
new= pd.DataFrame()
new

""


In [122]:
for i in range(2000, 2017):
    
    if i==2016:
        for k in range(1,4):
            if k==3:
                House[str(i) + 'q' + str(k)] = (House[str(i) + '-0' + str(int(k*3-2))] + House[str(i) + '-0' + str(int(k*3-1))])/2
                new[str(i) + 'q' + str(k)] = House[str(i) + 'q' + str(k)]
            else:
                House[str(i) + 'q' + str(k)] = (House[str(i) + '-0' + str(int(k*3-2))] + House[str(i) + '-0' + str(int(k*3-1))] + House[str(i) + '-0' + str(int(k*3))])/3
                new[str(i) + 'q' + str(k)] = House[str(i) + 'q' + str(k)]
    else:     
        for k in range(1,5):
            if k == 1 or k==2 or k==3:
                House[str(i) + 'q' + str(k)] = (House[str(i) + '-0' + str(int(k*3-2))] + House[str(i) + '-0' + str(int(k*3-1))] + House[str(i) + '-0' + str(int(k*3))])/3
                new[str(i) + 'q' + str(k)] = House[str(i) + 'q' + str(k)]
            else:
                House[str(i) + 'q' + str(k)] = (House[str(i) + '-' + str(int(k*3-2))] + House[str(i) + '-' + str(int(k*3-1))] + House[str(i) + '-' + str(int(k*3))])/3
                new[str(i) + 'q' + str(k)] = House[str(i) + 'q' + str(k)]

                

new.set_index([House['State'], House['RegionName']], inplace= True)
new

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [123]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return new
new.loc["Texas"].loc["Austin"].loc["2010q3"]

202266.66666666666

In [131]:
new['PriceRatio'] = new[Rec_start].div(new[min(GDP.apply(buttom,axis=1))[1]])
new

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3,PriceRatio
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0,1.072844
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0,1.134332
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0,1.055985
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0,1.006600
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0,1.151773
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0,1.298377
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0,1.090287
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0,1.067238
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0,1.099629


In [156]:
towns = pd.merge(new.reset_index(), df, indicator='flag', on = df.columns.tolist(), how= 'outer')
a = towns[['State', 'RegionName', 'flag', 'PriceRatio']]

unitown = a[a['flag']=='both']
notuni = a[a['flag'] == 'left_only']


,State,RegionName,flag,PriceRatio
0,New York,New York,left_only,1.072844
1,California,Los Angeles,left_only,1.134332
2,Illinois,Chicago,left_only,1.055985
3,Pennsylvania,Philadelphia,left_only,1.006600
4,Arizona,Phoenix,left_only,1.151773
8,California,San Jose,left_only,1.099629
9,Florida,Jacksonville,left_only,1.098698
10,California,San Francisco,left_only,1.085420
12,Michigan,Detroit,left_only,1.097615
15,North Carolina,Charlotte,left_only,1.059177


In [161]:
ttest = ttest_ind(unitown['PriceRatio'], notuni['PriceRatio'], nan_policy='omit')

In [164]:
p_value= ttest[1]
different = p_value < 0.01
if ttest[0]<=0:
    better = 'university town'
else:
    better = 'non-university town'
    
d = (different, p_value, better)

(True, 0.0059928062139885064, 'university town')

In [166]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return d
run_ttest()

(True, 0.0059928062139885064, 'university town')